# Purpose

### 2022-09-09
Test running queries in parallel with `dask`. Now that we'll run ANN for 250+ subreddits, running in a single thread could take a loooong time.


### 2022-08-01
Calculating precise nearest neighbors has become too expensive as we go over 40k subreddits. So instead let's calculate approx nearest neighbors (ANN). 

In this notebook we use [ANNOY](https://github.com/spotify/annoy).  Main reason for using annoy over FAISS is that annoy has official wheels in pypi, but FAISS only officially supports installation from conda. For now we don't want to depend on third-party wheels for FAISS b/c that can be messy to install & replicate in a VM. Maybe when we switch to kubeflow we can try FAISS.


# Notebook setup

In [1]:
%load_ext autoreload
%autoreload 2

In [152]:
from datetime import datetime
import gc
import os
import logging
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import seaborn as sns

import dask
from dask import dataframe as dd
from tqdm import tqdm

import mlflow
import hydra
import annoy

import subclu
from subclu.models.aggregate_embeddings import (
    AggregateEmbeddings, AggregateEmbeddingsConfig,
    load_config_agg_jupyter, get_dask_df_shape,
)
from subclu.models import aggregate_embeddings_pd

from subclu.utils import set_working_directory, get_project_subfolder
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric
)
from subclu.utils.mlflow_logger import MlflowLogger, save_pd_df_to_parquet_in_chunks
from subclu.eda.aggregates import (
    compare_raw_v_weighted_language
)
from subclu.utils.data_irl_style import (
    get_colormap, theme_dirl
)
from subclu.models.reshape_clusters_v050 import save_fpr_json


print_lib_versions([annoy, dask, hydra, mlflow, np, pd, plotly, sns, subclu])

python		v 3.7.10
===
annoy		v: 1.17.0
dask		v: 2021.06.0
hydra		v: 1.1.0
mlflow		v: 1.16.0
numpy		v: 1.19.5
pandas		v: 1.2.4
plotly		v: 4.14.3
seaborn		v: 0.11.1
subclu		v: 0.6.0


In [3]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()

# Set sqlite database as MLflow URI

In [4]:
# use new class to initialize mlflow
mlf = MlflowLogger(tracking_uri='sqlite')
mlflow.get_tracking_uri()

'sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-100-2021-04-28-djb-eda-german-subs/mlruns.db'

## Get list of experiments with new function

In [5]:
mlf.list_experiment_meta(output_format='pandas').tail(9)

,experiment_id,name,artifact_location,lifecycle_stage
30,30,v0.5.0_mUSE_clustering_test,gs://i18n-subreddit-clustering/mlflow/mlruns/30,active
31,31,v0.5.0_mUSE_clustering,gs://i18n-subreddit-clustering/mlflow/mlruns/31,active
32,32,v0.5.0_nearest_neighbors_test,gs://i18n-subreddit-clustering/mlflow/mlruns/32,active
33,33,v0.5.0_nearest_neighbors,gs://i18n-subreddit-clustering/mlflow/mlruns/33,active
34,34,v0.6.0_mUSE_aggregates_test,gs://i18n-subreddit-clustering/mlflow/mlruns/34,active
35,35,v0.6.0_mUSE_aggregates,gs://i18n-subreddit-clustering/mlflow/mlruns/35,active
36,36,v0.6.0_mUSE_clustering_test,gs://i18n-subreddit-clustering/mlflow/mlruns/36,active
37,37,v0.6.0_mUSE_clustering,gs://i18n-subreddit-clustering/mlflow/mlruns/37,active
38,38,v0.6.0_nearest_neighbors,gs://i18n-subreddit-clustering/mlflow/mlruns/38,active


## Get runs from embeddings aggregation jobs

Want to make sure we can load these artifacts for other jobs

In [6]:
%%time

df_mlf_runs =  mlf.search_all_runs(experiment_ids=[35])
df_mlf_runs.shape

CPU times: user 59.9 ms, sys: 52 µs, total: 60 ms
Wall time: 59.3 ms


(4, 43)

In [7]:
df_mlf_runs.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.memory_used_percent,metrics.df_v_post_comments-rows,metrics.time_fxn-df_subs_agg_c1,metrics.df_subs_agg_c1-cols,metrics.time_fxn-data_loading_time,metrics.df_posts_agg_c1-rows,metrics.df_subs_agg_c1-rows,metrics.cpu_count,metrics.memory_total,metrics.df_v_post_comments-cols,metrics.df_v_subs-rows,metrics.df_posts_agg_c1-cols,metrics.df_subs_agg_c1_uw-rows,metrics.time_fxn-full_aggregation_fxn_minutes,metrics.df_subs_agg_c1_uw-cols,metrics.df_v_subs-cols,metrics.memory_used,metrics.time_fxn-df_posts_agg_c1_no_delay,metrics.memory_free,metrics.time_fxn-df_subs_agg_c1_uw,params.mlflow_experiment,params.embeddings_subreddit_path,params.bucket_output,params.weight_subreddit_meta,params.mlflow_tracking_uri,params.weight_post_and_comments,params.host_name,params.agg_style,params.embeddings_bucket,params.cpu_count,params.memory_total,params.embeddings_post_and_comments_path,tags.mlflow.source.type,tags.mlflow.source.git.commit,tags.host_name,tags.mlflow.user,tags.mlflow.source.name
0,badc44b0e5ac467da14f710da0b410c6,35,FINISHED,gs://i18n-subreddit-clustering/mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts,2022-08-16 08:41:53.006000+00:00,2022-08-31 04:01:44.575000+00:00,0.536248,51906348.0,15.926672,515.0,3.698822,51906348.0,771760.0,96.0,1444961.0,515.0,771760.0,515.0,771760.0,820.674805,515.0,514.0,774858.0,544.288655,1197814.0,15.926672,v0.6.0_mUSE_aggregates,i18n_topic_model_batch/runs/20220811/subreddits/text/embedding/2022-08-11_082859,i18n-subreddit-clustering,0.15,sqlite,0.85,djb-100-2021-04-28-djb-eda-german-subs,dask_delayed,i18n-subreddit-clustering,96,1444961,i18n_topic_model_batch/runs/20220811/post_and_comment_text_combined/text_all/embedding/2022-08-11_084218,LOCAL,df6a30d80cfe36c1badb1531c7cbae7dd1046f21,djb-100-2021-04-28-djb-eda-german-subs,jupyter,/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py
1,ca79765b72c5428395b02926612d85fd,35,FINISHED,gs://i18n-subreddit-clustering/mlflow/mlruns/35/ca79765b72c5428395b02926612d85fd/artifacts,2022-08-16 08:41:31.162000+00:00,2022-08-31 03:13:27.187000+00:00,0.306086,51906348.0,NaN,NaN,3.719202,NaN,NaN,96.0,1444961.0,515.0,771760.0,NaN,NaN,NaN,NaN,514.0,442282.0,NaN,1175788.0,NaN,v0.6.0_mUSE_aggregates,i18n_topic_model_batch/runs/20220811/subreddits/text/embedding/2022-08-11_082859,i18n-subreddit-clustering,0.15,sqlite,0.85,djb-100-2021-04-28-djb-eda-german-subs,serial,i18n-subreddit-clustering,96,1444961,i18n_topic_model_batch/runs/20220811/post_and_comment_text_combined/text_all/embedding/2022-08-11_084218,LOCAL,df6a30d80cfe36c1badb1531c7cbae7dd1046f21,djb-100-2021-04-28-djb-eda-german-subs,jupyter,/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py
2,7552abcd785d4e229c6272aebf1beaf3,35,FAILED,gs://i18n-subreddit-clustering/mlflow/mlruns/35/7552abcd785d4e229c6272aebf1beaf3/artifacts,2022-08-16 08:35:25.596000+00:00,2022-08-16 08:40:12.627000+00:00,0.041416,11644466.0,NaN,NaN,0.734633,NaN,NaN,96.0,1444961.0,515.0,771760.0,NaN,NaN,NaN,NaN,514.0,59844.0,NaN,1281046.0,NaN,v0.6.0_mUSE_aggregates,i18n_topic_model_batch/runs/20220811/subreddits/text/embedding/2022-08-11_082859,i18n-subreddit-clustering,0.15,sqlite,0.85,djb-100-2021-04-28-djb-eda-german-subs,dask_delayed,i18n-subreddit-clustering,96,1444961,i18n_topic_model_batch/runs/20220811/post_and_comment_text_combined/text_all/embedding/2022-08-11_084218,LOCAL,df6a30d80cfe36c1badb1531c7cbae7dd1046f21,djb-100-2021-04-28-djb-eda-german-subs,jupyter,/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py
3,15b78241482e492cae90644e9733b50c,35,FAILED,gs://i18n-subreddit-clustering/mlflow/mlruns/35/15b78241482e492cae90644e9733b50c/artifacts,2022-08-16 07:37:37.787000+00:00,2022-08-16 07:43:09.168000+00:00,0.001953,NaN,NaN,NaN,NaN,NaN,NaN,96.0,1444961.0,NaN,771760.0,NaN,NaN,NaN,NaN,514.0,2822.0,NaN,1430384.0,NaN,None,i18n_topic_model_batch/runs/20220811/subreddits/text/embedding/2022-08-11_082859,None,0.15,None,0.85,djb-100-2021-

In [8]:
run_uuid = 'badc44b0e5ac467da14f710da0b410c6'

# Check run artifacts

In [9]:
l_artifacts_top_level = mlf.list_run_artifacts(
    run_id=run_uuid,
    only_top_level=True,
    verbose=True,
)
len(l_artifacts_top_level)

06:45:15 | INFO | "   219 <- Artifacts to check count"
06:45:15 | INFO | "   219 <- Artifacts clean count"
06:45:15 | INFO | "     5 <- Artifacts & folders at TOP LEVEL clean count"


5

In [10]:
l_artifacts_all = mlf.list_run_artifacts(
    run_id=run_uuid,
    only_top_level=False,
    verbose=False,
)
len(l_artifacts_all)

06:45:21 | INFO | "   219 <- Artifacts clean count"
06:45:21 | INFO | "     5 <- Artifacts & folders at TOP LEVEL clean count"


219

In [11]:
l_artifacts_top_level

['df_posts_agg_c1',
 'df_subs_agg_c1',
 'df_subs_agg_c1_ndjson',
 'df_subs_agg_c1_unweighted',
 'df_subs_agg_c1_unweighted_ndjson']

In [12]:
l_sub_c = [i for i in l_artifacts_all if 'df_subs_agg_c1' in i]
print(len(l_sub_c))
l_sub_c[:6]

14


['mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_subs_agg_c1/_common_metadata',
 'mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_subs_agg_c1/_metadata',
 'mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_subs_agg_c1/part.0.parquet',
 'mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_subs_agg_c1/part.1.parquet',
 'mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_subs_agg_c1/part.2.parquet',
 'mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_subs_agg_c1/part.3.parquet']

In [13]:
l_post_c = [i for i in l_artifacts_all if 'df_posts_agg_c1' in i]
print(len(l_post_c))
l_post_c[:6]

205


['mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_posts_agg_c1/_common_metadata',
 'mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_posts_agg_c1/_metadata',
 'mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_posts_agg_c1/part.0.parquet',
 'mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_posts_agg_c1/part.1.parquet',
 'mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_posts_agg_c1/part.10.parquet',
 'mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_posts_agg_c1/part.100.parquet']

# Load aggregated embeddings

use `gsutil` to download embeddings for posts b/c that can take a LONG time to download sequentially. `gsutil` makes parallel downloaidng much faster and reports download speeds above 500MB / s:
```bash
ents_sub_desc/part.67.parquet...
/ [2/197 files][ 61.7 GiB/ 75.4 GiB]  81% Done 632.0 MiB/s ETA 00:00:22
```

In [14]:
%%time

df_agg_sub_c = mlf.read_run_artifact(
    run_id=run_uuid,
    artifact_folder='df_subs_agg_c1',
    read_function='pd_parquet',
    verbose=False,
)
print(df_agg_sub_c.shape)

06:45:27 | INFO | "Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/mlflow/mlruns/35/badc44b0e5ac467da14f710da0b410c6/artifacts/df_subs_agg_c1"
100%|########################################| 14/14 [00:00<00:00, 44283.75it/s]
06:45:27 | INFO | "  Parquet files found:     4"
06:45:27 | INFO | "  Parquet files to use:     4"


(771760, 515)
CPU times: user 9.2 s, sys: 5.89 s, total: 15.1 s
Wall time: 7.88 s


In [15]:
df_agg_sub_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771760 entries, 0 to 771759
Columns: 515 entries, subreddit_id to embeddings_511
dtypes: float32(512), int64(1), object(2)
memory usage: 1.5+ GB


In [16]:
df_agg_sub_c.iloc[:5, :25]

,subreddit_id,subreddit_name,posts_for_embeddings_count,embeddings_0,embeddings_1,embeddings_2,embeddings_3,embeddings_4,embeddings_5,embeddings_6,embeddings_7,embeddings_8,embeddings_9,embeddings_10,embeddings_11,embeddings_12,embeddings_13,embeddings_14,embeddings_15,embeddings_16,embeddings_17,embeddings_18,embeddings_19,embeddings_20,embeddings_21
0,t5_1001tl,jewel_xo,1,-0.013827,-0.022239,0.049441,0.049470,-0.003573,0.040300,-0.017904,0.008067,-0.037719,-0.002597,0.008067,0.063312,0.014693,-0.042116,0.039357,-0.001491,0.068194,-0.022800,0.044375,0.017662,0.055988,-0.039419
1,t5_10029e,milkyhentai,1,-0.023227,-0.002677,0.031942,-0.010885,-0.023660,0.031216,0.044461,-0.024292,0.015413,0.047858,0.066368,0.076699,-0.040204,-0.004243,-0.048563,0.010120,0.024157,-0.020109,0.055920,0.015352,0.033357,0.010606
2,t5_1006k8,badwouldyourather,1,-0.032487,0.024979,-0.021948,0.039006,0.053261,0.037567,0.036113,0.011514,0.012002,0.020055,0.052276,0.025450,0.050676,-0.000008,-0.005012,0.000559,0.058759,-0.002293,0.010347,0.009482,0.024522,-0.023720
3,t5_100806,jojojosiah,2,0.004711,0.005103,0.037912,0.023591,0.024590,0.029586,-0.012185,-0.031729,-0.016308,0.063303,-0.015289,0.008682,0.008985,0.006243,-0.000484,0.015052,0.003276,0.002508,0.009955,-0.004335,0.001302,-0.016699
4,t5_1009a3,memesenespanol,380,0.003731,-0.013876,-0.003987,0.002683,-0.010202,0.038552,0.012759,0.016535,-0.056693,0.001183,0.009329,-0.005247,0.009630,-0.001513,0.000606,-0.004258,-0.005582,-0.002777,-0.001939,0.002463,-0.003895,-0.006126


In [17]:
df_agg_sub_c.iloc[-10:, :25]

,subreddit_id,subreddit_name,posts_for_embeddings_count,embeddings_0,embeddings_1,embeddings_2,embeddings_3,embeddings_4,embeddings_5,embeddings_6,embeddings_7,embeddings_8,embeddings_9,embeddings_10,embeddings_11,embeddings_12,embeddings_13,embeddings_14,embeddings_15,embeddings_16,embeddings_17,embeddings_18,embeddings_19,embeddings_20,embeddings_21
771750,t5_6qo047,scienceno488,0,0.049827,0.008324,0.033611,0.032743,0.025569,0.084280,-0.016352,-0.054408,-0.030566,-0.041512,-0.073808,0.016681,-0.042703,0.033103,-0.078955,-0.012576,-0.057177,-0.055155,-0.023454,-0.001195,-0.053379,0.017656
771751,t5_6phqtd,jessivann,0,-0.030140,-0.029670,0.020029,0.082169,0.044654,0.040252,-0.059091,-0.060705,-0.063691,-0.000536,0.060795,0.091693,-0.052438,-0.042100,-0.031989,0.038370,-0.013700,-0.020486,0.059314,0.009449,0.082631,-0.055906
771752,t5_6ttq8w,emiii,0,-0.014652,-0.025286,0.043648,0.031067,0.015594,0.049370,-0.043500,-0.038448,-0.079371,0.045315,0.055310,0.089274,-0.030252,0.011965,-0.010014,-0.002786,0.049631,0.012006,0.053870,0.002287,0.051208,0.021202
771753,t5_6hetta,tinomantana,0,-0.039023,0.003490,0.019000,0.030172,0.045525,0.027864,-0.062640,-0.071491,-0.060166,0.022435,0.082675,0.084102,-0.073745,0.023707,-0.021793,-0.006323,0.015783,-0.057229,0.059388,0.006303,0.070736,-0.043325
771754,t5_6t85bh,twistedspun,0,-0.041944,0.011073,0.008569,0.025391,0.049350,0.047871,-0.018863,0.007396,-0.059508,0.070213,0.088125,0.106481,-0.048567,0.022983,-0.023090,-0.011047,0.042330,-0.022907,0.015026,0.017319,0.037380,0.012689
771755,t5_wi2zh,autocadmemes,0,-0.031634,-0.032420,-0.076026,-0.027858,0.026071,-0.078470,0.050454,0.065865,-0.034230,-0.037524,0.073746,-0.062604,-0.046956,-0.072842,-0.072843,0.040903,0.066408,0.024241,-0.073683,0.017086,-0.057149,0.003010
771756,t5_6rhnln,gamestopcensorship,0,-0.033220,-0.046152,0.040127,-0.071822,0.037267,0.045015,-0.014639,0.002809,-0.014747,-0.030380,0.042313,-0.050563,-0.068255,-0.057389,0.008784,-0.008465,0.006220,-0.029101,-0.070836,-0.013270,0.021226,0.000765
771757,t5_6pg8i2,coffeepoblog,0,-0.023961,-0.002170,0.044053,-0.000543,0.048995,0.034303,-0.014573,0.051975,-0.068334,-0.032103,-0.026937,-0.056592,-0.054808,0.002123,-0.082676,0.071959,-0.061371,-0.021760,0.016584,-0.029590,0.019676,-0.072884
771758,t5_6cv9bz,threadtreatment,0,-0.020639,0.030801,-0.007754,0.075224,0.028055,0.026198,0.027853,0.057297,-0.058218,0.045436,0.031521,0.027640,-0.053154,0.035169,-0.021854,-0.052122,0.019887,-0.027726,-0.095366,-0.017542,-0.018356,-0.046369
771759,t5_4la7jo,spongebobcringememes,0,-0.004059,0.013185,-0.059028,0.054473,0.048189,0.025800,0.028421,0.038999,-0.092010,0.065643,0.035775,-0.049399,-0.010349,-0.042322,-0.045216,-0.043074,0.044714,-0.033876,0.050600,0.025916,0.047870,0.000057


# Filter subreddits to use in ANN index

In a previous version we only kept subs that had embeddings AND clustering data. 
<br>Now that we cover 700k subreddits for v0.6.0, we need to be more thoughtfula bout how we'll select which subs to keep for ANN.

For v0.6.0 we'll keep only subs that have 5+ posts in L90 days. From this mode dashboard we expect that number to be around 250k subreddits.

Mode Dashboard: https://app.mode.com/reddit/reports/e6cde33162c4 

## Load metdata to apply other filters [optional]

If we want to filter subreddits based on other data, we'll need to pull data from mlflow or BigQuery.


In [18]:
# run_id_final_model = ''

In [19]:
# l_artifacts_top_level = mlf.list_run_artifacts(
#     run_id=run_id_final_model,
#     only_top_level=True,
#     verbose=True,
# )
# len(l_artifacts_top_level)

In [20]:
# l_artifacts_all = mlf.list_run_artifacts(
#     run_id=run_id_final_model,
#     only_top_level=False,
#     verbose=False,
# )
# len(l_artifacts_all)

In [21]:
# l_artifacts_top_level

In [22]:
# l_sub_c = [i for i in l_artifacts_all if 'df_labels' in i]
# print(len(l_sub_c))
# l_sub_c[:6]

In [23]:
# df_labels = mlf.read_run_artifact(
#     run_id=run_id_final_model,
#     artifact_folder='df_labels',
#     read_function='pd_parquet',
#     verbose=False,
# )
# print(df_labels.shape)

In [24]:
# df_labels.iloc[:5, :15]

## Apply filters

In v0.6.0, we already have the number of posts for embedding in the embedding file, so we don't need to load additional files to apply filters

In [25]:
# use df_pc_counts because it has the counts for post+comment after filtering for length
value_counts_and_pcts(
  pd.cut(
      df_agg_sub_c['posts_for_embeddings_count'],
      bins=[-1, 0, 1, 2, 3, 4, 5, 49, np.inf],
      labels=[
        "00 posts", "01 post", '02 posts', '03 posts',
        '04 posts', '05 posts'
        , '06-49 posts', '50+ posts'
      ]
  ).rename('posts_with_len_3+'),
  sort_index=True,
  add_col_prefix=False,
  count_type='subreddits',
  sort_index_ascending=False,
  cumsum_count=True,
  reset_index=True,
).hide_index().set_caption(f"<h4 align='left'>Post distribution for subreddits with 1 view & 1 attempted post in L90-days</h4>")

posts_with_len_3+,subreddits_count,percent_of_subreddits,cumulative_sum_of_subreddits,cumulative_percent_of_subreddits
50+ posts,"72,510",9.4%,"72,510",9.4%
06-49 posts,"154,858",20.1%,"227,368",29.5%
05 posts,"23,205",3.0%,"250,573",32.5%
04 posts,"33,084",4.3%,"283,657",36.8%
03 posts,"56,898",7.4%,"340,555",44.1%
02 posts,"125,070",16.2%,"465,625",60.3%
01 post,"240,338",31.1%,"705,963",91.5%
00 posts,"65,797",8.5%,"771,760",100.0%


In [26]:
df_agg_sub_c_raw = df_agg_sub_c.copy()

df_agg_sub_c = df_agg_sub_c[df_agg_sub_c['posts_for_embeddings_count'] >= 5]
df_agg_sub_c.shape

(250573, 515)

# Build annoy index

I created a custom `AnnoyIndex` class with some extra methods to create outputs & (and calculate cosine distance) for BigQuery.

In [27]:
from subclu.models.nn_annoy import AnnoyIndex

In [167]:
%%time

index_cols = ['subreddit_id', 'subreddit_name']
l_embedding_cols = [c for c in df_agg_sub_c.columns if c.startswith('embeddings_')]
n_trees = 20
n_sample_subs = 9000
nn_index = AnnoyIndex(
    df_agg_sub_c[l_embedding_cols + index_cols].sample(n=n_sample_subs, random_state=42),
#     df_agg_sub_c[l_embedding_cols + index_cols],
    index_cols=index_cols,
    metric='angular',
    n_trees=n_trees,
)

nn_index.build()

CPU times: user 1.12 s, sys: 205 ms, total: 1.33 s
Wall time: 938 ms


## Test `search_k`
`search_k=-1` will search all trees and get the most accurate results but it will take longer to compute.

Even with small changes we can see in the examples below that there is a time difference and sometimes even in the top10 results we will miss a neighbor when we set k<=3 (only search k=3 -> only search 3 trees).

In [168]:
%%time

n_test_i = 212
nn_index.get_top_n_by_item(n_test_i, k=9, search_k=-1, include_distances=True)

CPU times: user 16.7 ms, sys: 0 ns, total: 16.7 ms
Wall time: 15.5 ms


,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance
0,t5_2qm9d,cfb,0,t5_2qm9d,cfb,0.000000
1,t5_2qm9d,cfb,1,t5_2ra1q,hokies,0.636392
2,t5_2qm9d,cfb,2,t5_32qmp,gatorsfootball,0.651490
3,t5_2qm9d,cfb,3,t5_2xvqe,cfbanalysis,0.690349
4,t5_2qm9d,cfb,4,t5_2srai,floridastate,0.739856
5,t5_2qm9d,cfb,5,t5_2wjlu,12thman,0.759137
6,t5_2qm9d,cfb,6,t5_2un8g,tcufootball,0.759586
7,t5_2qm9d,cfb,7,t5_2secp,sportingkc,0.802073
8,t5_2qm9d,cfb,8,t5_1221xi,deepdishfootball,0.819388


In [169]:
%%time
nn_index.get_top_n_by_item(n_test_i, k=9, search_k=1, include_distances=True)

CPU times: user 15.3 ms, sys: 107 µs, total: 15.4 ms
Wall time: 14.4 ms


,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance
0,t5_2qm9d,cfb,0,t5_2qm9d,cfb,0.000000
1,t5_2qm9d,cfb,1,t5_2ra1q,hokies,0.636392
2,t5_2qm9d,cfb,2,t5_32qmp,gatorsfootball,0.651490
3,t5_2qm9d,cfb,3,t5_2xvqe,cfbanalysis,0.690349
4,t5_2qm9d,cfb,4,t5_2srai,floridastate,0.739856
5,t5_2qm9d,cfb,5,t5_2wjlu,12thman,0.759137
6,t5_2qm9d,cfb,6,t5_2un8g,tcufootball,0.759586
7,t5_2qm9d,cfb,7,t5_2secp,sportingkc,0.802073
8,t5_2qm9d,cfb,8,t5_1221xi,deepdishfootball,0.819388


In [170]:
top_k_test_ = 20
cols_drop_ = ['subreddit_id_a', 'subreddit_id_b', 'distance']
cols_append_ = ['subreddit_name_b',]
df_compare_sk = nn_index.get_top_n_by_item(
    n_test_i, k=top_k_test_, search_k=-1, include_distances=True
).drop(cols_drop_, axis=1)

for k_ in [int(0.998 * n_trees), int(0.85 * n_trees), 
           int(0.5 * n_trees), min([200, int(0.1 * n_trees)]),
           1]:
    df_compare_sk = pd.concat(
        [
            df_compare_sk,
            nn_index.get_top_n_by_item(
                n_test_i, k=top_k_test_, search_k=k_, include_distances=True
            )[cols_append_].rename(columns={c: f"{c}_{k_}" for c in df_compare_sk.columns})
        ],
        axis=1,
    )
df_compare_sk

,subreddit_name_a,distance_rank,subreddit_name_b,subreddit_name_b_19,subreddit_name_b_17,subreddit_name_b_10,subreddit_name_b_2,subreddit_name_b_1
0,cfb,0,cfb,cfb,cfb,cfb,cfb,cfb
1,cfb,1,hokies,hokies,hokies,hokies,hokies,hokies
2,cfb,2,gatorsfootball,gatorsfootball,gatorsfootball,gatorsfootball,gatorsfootball,gatorsfootball
3,cfb,3,cfbanalysis,cfbanalysis,cfbanalysis,cfbanalysis,cfbanalysis,cfbanalysis
4,cfb,4,floridastate,floridastate,floridastate,floridastate,floridastate,floridastate
5,cfb,5,12thman,12thman,12thman,12thman,12thman,12thman
6,cfb,6,tcufootball,tcufootball,tcufootball,tcufootball,tcufootball,tcufootball
7,cfb,7,sportingkc,sportingkc,sportingkc,sportingkc,sportingkc,sportingkc
8,cfb,8,deepdishfootball,deepdishfootball,deepdishfootball,deepdishfootball,deepdishfootball,deepdishfootball
9,cfb,9,mlrugby,mlrugby,mlrugby,mlrugby,mlrugby,mlrugby


## Get df with all items

For 80k subreddits it took:

Test a few values of k_search.
In testing, it looks like the total time is about the same for different values of K... looks like most of the time is spent doing manipulations with pandas, so k is almost not important :hmm:


```bash
100%|██████████| 81973/81973 [1:17:02<00:00, 17.73it/s]
17:07:23 | INFO | "(8115327, 7) <- df_top_items shape"
```

In [128]:
pct_trees_ = 0.85
search_k_ = int(pct_trees_ * n_trees)
print(f"Percent of trees: {pct_trees_} | search_k={search_k_}")
df_nn_top = nn_index.get_top_n_by_item_all(
    k=50,
    search_k=search_k_,
    include_distances=True,
    append_i=True,
)

Percent of trees: 0.85 | search_k=25


100%|██████████| 9000/9000 [01:36<00:00, 92.83it/s]
07:55:00 | INFO | "(441000, 7) <- df_top_items shape"


In [130]:
print(f"search all (k_search=-1)")
df_nn_top = nn_index.get_top_n_by_item_all(
    k=50,
    search_k=-1, 
    include_distances=True,
    append_i=True,
)

search all (k_search=-1)


100%|██████████| 9000/9000 [01:38<00:00, 91.23it/s]
07:57:03 | INFO | "(441000, 7) <- df_top_items shape"


In [171]:
%%time

pct_trees_ = 0.5
search_k_ = int(pct_trees_ * n_trees)
print(f"Percent of trees: {pct_trees_} | search_k={search_k_}")
df_nn_top = nn_index.get_top_n_by_item_all(
    k=50,
    search_k=search_k_,
    include_distances=True,
    append_i=True,
)

Percent of trees: 0.5 | search_k=10


100%|##########| 9000/9000 [01:36<00:00, 93.23it/s]
08:41:53 | INFO | "(441000, 7) <- df_top_items shape"


CPU times: user 1min 40s, sys: 118 ms, total: 1min 40s
Wall time: 1min 40s


## Test dask delayed

How much faster will it be if we run more things in parallel using dask?

LOL - it's actually slower :cry_cool:

In [172]:
%%time

pct_trees_ = 0.5
search_k_ = int(pct_trees_ * n_trees)
print(f"Percent of trees: {pct_trees_} | search_k={search_k_}")
df_nn_top_dask = nn_index.get_top_n_by_item_all_dask(
    k=50,
    search_k=search_k_,
    include_distances=True,
    append_i=True,
)

08:41:53 | INFO | "Creating DAG with dask..."


Percent of trees: 0.5 | search_k=10


100%|##########| 9000/9000 [00:01<00:00, 7834.83it/s]
08:41:55 | INFO | "Start Computing DAG..."
08:44:12 | INFO | "DONE Computing DAG "
08:44:12 | INFO | "(441000, 7) <- df_top_items shape"


CPU times: user 2min 55s, sys: 27.8 s, total: 3min 23s
Wall time: 2min 18s


## Test going back to basics
The raw ANNOY calls are super fast and only return lists (or tuples of lists).  So start from that

In [174]:
nn_index.index.get_nns_by_item(
    1,
    5,
    search_k=search_k,
    include_distances=False
)

[1, 5858, 915, 7061, 7424]

In [175]:
nn_index.index.get_nns_by_item(
    1,
    5,
    search_k=search_k,
    include_distances=True
)

([1, 5858, 915, 7061, 7424],
 [0.0,
  0.8734745383262634,
  0.876518964767456,
  0.9028976559638977,
  0.9035140872001648])

In [252]:
%%time

from typing import Tuple

d_topk_nn = dict()
top_k_ = 5
search_k = -1

for i in tqdm(range(nn_index.n_rows)):
    i_nn: Tuple[int, float] = nn_index.index.get_nns_by_item(
        1,
        top_k_ + 1,
        search_k=search_k,
        include_distances=True
    )
    # we need to skip the first item because it's always the self (index number)
    d_topk_nn[i] = {
        'nn_ix': i_nn[0][1:],
        'distance': i_nn[1][1:],
        'distance_rank': [v for v in range(1, top_k_ + 1)]
    }
df_nn_top_ix1 = pd.concat(
    [pd.DataFrame(d_topk_nn[i_], index=[i_] * top_k_) for i_ in range(nn_index.n_rows)]
)

100%|██████████| 9000/9000 [00:00<00:00, 13914.02it/s]


CPU times: user 6.38 s, sys: 185 ms, total: 6.56 s
Wall time: 6.37 s


In [250]:
%%time

_ = pd.concat(
    [pd.DataFrame(d_topk_nn[i_], index=[i_] * top_k_) for i_ in range(nn_index.n_rows)]
)

CPU times: user 5.76 s, sys: 252 ms, total: 6.01 s
Wall time: 5.63 s


In [251]:
%%time

l_topk_dfs = list()
for i_ in range(nn_index.n_rows):
    l_topk_dfs.append(
        pd.DataFrame(d_topk_nn[i_], index=[i_] * top_k_)
    )
        
_ = pd.concat(l_topk_dfs)

CPU times: user 6.26 s, sys: 263 ms, total: 6.52 s
Wall time: 5.93 s


In [245]:
%%time

from typing import Tuple

# d_topk_nn = dict()
top_k_ = 5
search_k = -1
l_topk_dfs = list()


for i in tqdm(range(nn_index.n_rows)):
    i_nn: Tuple[int, float] = nn_index.index.get_nns_by_item(
        1,
        top_k_ + 1,
        search_k=search_k,
        include_distances=True
    )
    # we need to skip the first item because it's always the self (index number)
    
    # test converting this df creation into a dask.delayed job
    l_topk_dfs.append(
        pd.DataFrame(
            {
                'seed_ix': [i] * top_k_,
                'nn_ix': i_nn[0][1:],
                'distance': i_nn[1][1:],
                'distance_rank': [v for v in range(1, top_k_ + 1)]
            },
        )
    )
df_top_nn_fast = pd.concat(l_topk_dfs, ignore_index=True)
df_top_nn_fast.shape

100%|██████████| 9000/9000 [00:05<00:00, 1764.84it/s]


CPU times: user 6.93 s, sys: 59.3 ms, total: 6.98 s
Wall time: 6.97 s


(45000, 4)

### Dask overhead is too much
Looks like even with the simplified example dask adds too much overhead -- it almost doubles the time

In [248]:
%%time

from typing import Tuple

# d_topk_nn = dict()
top_k_ = 5
search_k = -1
l_topk_dfs = list()


for i in tqdm(range(nn_index.n_rows)):
    i_nn: Tuple[int, float] = nn_index.index.get_nns_by_item(
        1,
        top_k_ + 1,
        search_k=search_k,
        include_distances=True
    )
    # we need to skip the first item because it's always the self (index number)
    
    # test converting this df creation into a dask.delayed job
    l_topk_dfs.append(
        dask.delayed(pd.DataFrame)(
            {
                'seed_ix': [i] * top_k_,
                'nn_ix': i_nn[0][1:],
                'distance': i_nn[1][1:],
                'distance_rank': [v for v in range(1, top_k_ + 1)]
            },
        )
    )
df_top_nn_delayed = dask.delayed(pd.concat)(l_topk_dfs, ignore_index=True)
logging.info(f"start computing dag...")
df_top_nn_dask = df_top_nn_delayed.compute()
df_top_nn_dask.shape

100%|██████████| 9000/9000 [00:02<00:00, 3756.51it/s]
10:12:36 | INFO | "start computing dag..."


CPU times: user 12.2 s, sys: 813 ms, total: 13 s
Wall time: 11.6 s


(45000, 4)

In [226]:
pd.DataFrame(d_topk_nn[1], index=[1] * top_k_)

,nn_ix,distance,distance_rank
1,915,0.876519,1
1,7061,0.902898,2
1,7424,0.903514,3
1,4272,0.904678,4
1,3358,0.905303,5


## Check some example outputs

In [131]:
df_nn_top.head()

,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance,cosine_similarity
1,t5_5ymqlf,amatureslags,1,t5_2x06w4,exhib,0.401948,0.919219
2,t5_5ymqlf,amatureslags,2,t5_3gsee4,thekyliecupcake,0.412681,0.914847
3,t5_5ymqlf,amatureslags,3,t5_67us9u,teenhottness,0.420998,0.911381
4,t5_5ymqlf,amatureslags,4,t5_6jmja1,nakedporno,0.431574,0.906872
5,t5_5ymqlf,amatureslags,5,t5_6lk8ak,sofiagomezonlyfriends,0.434595,0.905564


In [132]:
(
    df_nn_top[df_nn_top['subreddit_name_a'] == 'ich_iel']
    .head(15)
)

,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance,cosine_similarity


In [133]:
(
    df_nn_top[df_nn_top['subreddit_name_a'] == 'vegetarischde']
    .head(15)
)

,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance,cosine_similarity


In [134]:
(
    df_nn_top[df_nn_top['subreddit_name_a'] == 'antivegan']
    .head(15)
)

,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance,cosine_similarity


In [135]:
(
    df_nn_top[df_nn_top['subreddit_name_a'] == 'mexico']
    .head(15)
)

,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance,cosine_similarity


In [136]:
(
    df_nn_top[df_nn_top['subreddit_name_a'] == 'de']
    .head(15)
)

,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance,cosine_similarity


# Convert plain distance to cosine similarity
from [github](https://github.com/spotify/annoy/issues/112#issuecomment-686513356)
```
cosine_similarity = 1 - cosine_distance^2/2
```

In [139]:
%%time

col_cosine_similarity = 'cosine_similarity'

if col_cosine_similarity not in df_nn_top:
    df_nn_top[col_cosine_similarity] = (
        1 -
        (df_nn_top['distance'] ** 2) / 2
    )

CPU times: user 28 µs, sys: 1 µs, total: 29 µs
Wall time: 33.9 µs


In [140]:
df_nn_top.head()

,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance,cosine_similarity
1,t5_5ymqlf,amatureslags,1,t5_2x06w4,exhib,0.401948,0.919219
2,t5_5ymqlf,amatureslags,2,t5_3gsee4,thekyliecupcake,0.412681,0.914847
3,t5_5ymqlf,amatureslags,3,t5_67us9u,teenhottness,0.420998,0.911381
4,t5_5ymqlf,amatureslags,4,t5_6jmja1,nakedporno,0.431574,0.906872
5,t5_5ymqlf,amatureslags,5,t5_6lk8ak,sofiagomezonlyfriends,0.434595,0.905564


In [142]:
(
    df_nn_top[df_nn_top['subreddit_name_a'] == 'ich_iel']
    .head(15)
)

,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance,cosine_similarity


In [143]:
(
    df_nn_top[df_nn_top['subreddit_name_a'] == 'cfb']
    .head(15)
)

,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance,cosine_similarity
10601,t5_2qm9d,cfb,1,t5_2ra1q,hokies,0.636392,0.797502
10602,t5_2qm9d,cfb,2,t5_32qmp,gatorsfootball,0.651490,0.787781
10603,t5_2qm9d,cfb,3,t5_2xvqe,cfbanalysis,0.690349,0.761709
10604,t5_2qm9d,cfb,4,t5_2srai,floridastate,0.739856,0.726306
10605,t5_2qm9d,cfb,5,t5_2wjlu,12thman,0.759137,0.711855
10606,t5_2qm9d,cfb,6,t5_2un8g,tcufootball,0.759586,0.711514
10607,t5_2qm9d,cfb,7,t5_2secp,sportingkc,0.802073,0.678339
10608,t5_2qm9d,cfb,8,t5_1221xi,deepdishfootball,0.819388,0.664301
10609,t5_2qm9d,cfb,9,t5_3j3bl,mlrugby,0.823590,0.660850
10610,t5_2qm9d,cfb,10,t5_lxntg,intermiami,0.847162,0.641158


In [141]:
(
    df_nn_top[df_nn_top['subreddit_name_a'] == 'finanzen']
    .head(15)
)

,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance,cosine_similarity


# Add dt/pt column & metadata columns
- ann dt

In [144]:
df_nn_top['ann_dt'] = pd.to_datetime(pd.to_datetime(datetime.utcnow()).date())

In [146]:
df_nn_top['model_version'] = '0.6.0 test'

In [148]:
df_nn_top.tail()

,subreddit_id_a,subreddit_name_a,distance_rank,subreddit_id_b,subreddit_name_b,distance,cosine_similarity,ann_dt,model_version
449995,t5_2tk05,nottimanderic,45,t5_2to51,oddfuture,0.770649,0.703050,2022-09-09,0.6.0 test
449996,t5_2tk05,nottimanderic,46,t5_6rf5sr,youtube100k,0.773699,0.700695,2022-09-09,0.6.0 test
449997,t5_2tk05,nottimanderic,47,t5_2gueoh,ichikanito,0.777590,0.697677,2022-09-09,0.6.0 test
449998,t5_2tk05,nottimanderic,48,t5_53yh9q,animereels,0.777949,0.697398,2022-09-09,0.6.0 test
449999,t5_2tk05,nottimanderic,49,t5_6s5qt9,sbe_podcast,0.777985,0.697369,2022-09-09,0.6.0 test


In [149]:
df_nn_top.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 441000 entries, 1 to 449999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   subreddit_id_a     441000 non-null  object        
 1   subreddit_name_a   441000 non-null  object        
 2   distance_rank      441000 non-null  int64         
 3   subreddit_id_b     441000 non-null  object        
 4   subreddit_name_b   441000 non-null  object        
 5   distance           441000 non-null  float64       
 6   cosine_similarity  441000 non-null  float64       
 7   ann_dt             441000 non-null  datetime64[ns]
 8   model_version      441000 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 33.6+ MB


# Save to local & log to Mlflow

Instead of saving it to random location in GCS, save artifact locally & then log it to mlflow job as a new artifact.

Make sure to append a timestamp in case we try different ANN approaches


In [153]:
manual_model_timestamp = datetime.utcnow().strftime('%Y-%m-%d_%H%M%S')
path_this_model = get_project_subfolder(
    f"data/models/ann/manual_v060_{manual_model_timestamp}"
)
Path.mkdir(path_this_model, parents=True, exist_ok=True)
path_this_model

PosixPath('/home/jupyter/subreddit_clustering_i18n/data/models/ann/manual_v060_2022-09-09_080702')

In [155]:
%%time

save_pd_df_to_parquet_in_chunks(
    df_nn_top,
    path_this_model,
    write_index=False
)

08:10:38 | INFO | "Converting pandas to dask..."
08:10:38 | INFO | "   158.6 MB <- Memory usage"
08:10:38 | INFO | "       1	<- target Dask partitions	  350.0 <- target MB partition size"


CPU times: user 595 ms, sys: 44.2 ms, total: 639 ms
Wall time: 676 ms


# Save df to GCS

From here we should be able to share & create a BigQuery table

In [ ]:
BREAK

In [66]:
import pyarrow as pa

In [86]:
gs_nearest_neighbors_root = 'gs://i18n-subreddit-clustering/data/models/nearest_neighbors'
gs_this_model = f"{gs_nearest_neighbors_root}/manual_model_{datetime.utcnow().strftime('%Y-%m-%d_%H%M%S')}"
gs_this_model

'gs://i18n-subreddit-clustering/data/models/nearest_neighbors/manual_model_2022-08-01_181632'

In [87]:
%%time
shape_ = df_nn_top.shape

df_nn_top.to_parquet(
    f"{gs_this_model}/df_nearest_neighbors_top_full-{shape_[0]}_by_{shape_[1]}.parquet"
)

CPU times: user 5.09 s, sys: 457 ms, total: 5.54 s
Wall time: 10.1 s


## Create schema for BigQuery parquet file

We'll save 1 file with all the raw data and a different file with the data needed to create a BQ table.

We'll also create a schema for the BQ file so that BQ can display some metadata about the tables

In [137]:
from google.cloud import bigquery


l_cols_for_bq = [
    'subreddit_id_a',
    'subreddit_name_a',
    'distance_rank',
    'subreddit_id_b',
    'subreddit_name_b',
    'cosine_similarity',
    'ann_dt',
    'model_version',
]

l_bq_json_schema = [
    bigquery.SchemaField(
        mode="REQUIRED",
        name="subreddit_id_a",
        field_type="STRING",
        description="Seed subreddit ID. Seed = subreddit for which you want to find N most similar subs"
    ),
    bigquery.SchemaField(
        mode="REQUIRED",
        name="subreddit_name_a",
        field_type="STRING",
        description="Seed subreddit name"
    ),
    bigquery.SchemaField(
        mode="NULLABLE",
        name="distance_rank",
        field_type="INTEGER",
        description="Rank for how closse subreddit_A is subreddit_B. 1=closest, 2=2nd closest"
    ),
    bigquery.SchemaField(
        mode="REQUIRED",
        name="subreddit_id_b",
        field_type="STRING",
        description="Subreddit ID for most similar subs to subreddit_a"
    ),
    bigquery.SchemaField(
        mode="REQUIRED",
        name="subreddit_name_b",
        field_type="STRING",
        description="Subreddit B name"
    ),
    bigquery.SchemaField(
        mode="REQUIRED",
        name="cosine_similarity",
        field_type="FLOAT",
        description="Similarity between sub-a & sub-b. 1=identical, 0=not related, -1=opposite"
    ),
    bigquery.SchemaField(
        mode="NULLABLE",
        name="ann_dt",
        field_type="TIMESTAMP",
        description="Date that approximate nearest neighbors file was created"
    ),
    bigquery.SchemaField(
        mode="NULLABLE",
        name="model_version",
        field_type="STRING",
        description="Version for CAU model that created the embeddings to compare subreddit distance"
    )
]
d_metadata_description = dict()
for sf_ in l_bq_json_schema:
    d_metadata_description[sf_.name] = sf_.description

pa_bq_schema = pa.schema(
    [
        pa.field(
            'subreddit_id_a', pa.string(),
            nullable=False,
            # This metadata doesn't get ready by BigQuery :/
            # metadata={'description': "Seed subreddit ID. Seed = subreddit for which you want to find N most similar subs"},
        ),
        pa.field(
            'subreddit_name_a', pa.string(),
        ),
        ('distance_rank', pa.int32()),
        ('subreddit_id_b', pa.string()),
        ('subreddit_name_b', pa.string()),
        ('cosine_similarity', pa.float64()),
        ('ann_dt', pa.timestamp('ns')),
        ('model_version', pa.string()),
    ],
    metadata=d_metadata_description,
)
pa_bq_schema

subreddit_id_a: string not null
subreddit_name_a: string
distance_rank: int32
subreddit_id_b: string
subreddit_name_b: string
cosine_similarity: double
ann_dt: timestamp[ns]
model_version: string
-- schema metadata --
subreddit_id_a: 'Seed subreddit ID. Seed = subreddit for which you want t' + 26
subreddit_name_a: 'Seed subreddit name'
distance_rank: 'Rank for how closse subreddit_A is subreddit_B. 1=closest' + 15
subreddit_id_b: 'Subreddit ID for most similar subs to subreddit_a'
subreddit_name_b: 'Subreddit B name'
cosine_similarity: 'Similarity between sub-a & sub-b. 1=identical, 0=not ' + 20
ann_dt: 'Date that approximate nearest neighbors file was created'
model_version: 'Version for CAU model that created the embeddings to comp' + 22

In [133]:
# pa_bq_schema.metadata

In [138]:
%%time
shape_ = df_nn_top[l_cols_for_bq].shape
path_bq = f"{gs_this_model}/df_nearest_neighbors_top_bigquery-{shape_[0]}_by_{shape_[1]}.parquet"
print(path_bq)

df_nn_top[l_cols_for_bq].to_parquet(
    path_bq,
    engine='pyarrow',
    schema=pa_bq_schema,
)

gs://i18n-subreddit-clustering/data/models/nearest_neighbors/manual_model_2022-08-01_181632/df_nearest_neighbors_top_bigquery-8115327_by_8.parquet
CPU times: user 5.22 s, sys: 413 ms, total: 5.63 s
Wall time: 8.29 s


# Upload to BQ

using `bq load` won't work with a JSON schema in BQ.

Instead, let's try using the python client. NOTE: we'll need to get the right authentication in the VM that has the correct read & write access, e.g.,:
```bash
# login
gcloud auth application-default login

# logout
gcloud auth application-default revoke
```

In [142]:
## This works, but won't read the "description" or other nice metadata from parquet files

# !bq load \
#     --source_format=PARQUET \
#     --project_id=reddit-employee-datasets \
#     david_bermejo.subclu_v0050_subreddit_distances_c_top_100 \
#     "gs://i18n-subreddit-clustering/data/models/nearest_neighbors/manual_model_2022-08-01_181632/df_nearest_neighbors_top_bigquery-8115327_by_8.parquet"

In [141]:
%%time

client = bigquery.Client()

# Set table_id to the ID of the table to create.
table_id = "reddit-employee-datasets.david_bermejo.subclu_v0050_subreddit_distances_c_top_100"

# WRITE_TRUNCATE -> replace
# WRITE_APPEND -> append data
job_config = bigquery.LoadJobConfig(
    schema=l_bq_json_schema,
    source_format=bigquery.SourceFormat.PARQUET,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)
uri = "gs://i18n-subreddit-clustering/data/models/nearest_neighbors/manual_model_2022-08-01_181632/df_nearest_neighbors_top_bigquery-8115327_by_8.parquet"

load_job = client.load_table_from_uri(
    uri,
    table_id,
    location="US",  # Must match the destination dataset location.
    job_config=job_config,
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))


Loaded 8115327 rows.
CPU times: user 46.7 ms, sys: 124 ms, total: 171 ms
Wall time: 1min 8s


# Appendix

In [18]:
# %%time
# # use gsutil to download post-level embeddings b/c it'll be much faster to run it in parallel

# remote_key =  'mlflow/mlruns/29/bfe6cbd59a21480c8c2b9923a3a9cbd6/artifacts/df_subs_agg_c1'

# # Need to remove the last part of the local path otherwise we'll get duplicate subfolders:
# #. top/2021-12-14/2021-12-14 instead of top/2021-12-14
# local_f = f"/home/jupyter/subreddit_clustering_i18n/data/local_cache/{'/'.join(remote_key.split('/')[:-1])}"
# Path(local_f).mkdir(parents=True, exist_ok=True)
# remote_gs_path = f"gs://i18n-subreddit-clustering/{remote_key}"
# print(f"Remote path:\n  {remote_gs_path}")
# print(f"Local path:\n  {local_f}")

# # `-n` flag means "no clober", so it should skip existing files (only copy new files)
# !gsutil -m cp -r -n $remote_gs_path $local_f

In [21]:
## We'll do posts in a separate notebook
# %%time

# df_agg_posts_c = mlf.read_run_artifact(
#     run_id=run_uuid,
#     artifact_folder='df_post_level_agg_c_post_comments_sub_desc',
#     read_function='pd_parquet',
#     verbose=False,
# )
# print(df_agg_posts_c.shape)